### 1. Setup enviroment

https://www.naturalistic-neuroimaging-database.org/annotations#Header8

#### Libs

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import tempfile
from tqdm import tqdm
from moviepy.editor import *
from IPython.display import Video, clear_output, HTML

#### Data

In [ ]:
# little_miss_sunshine annotation has delay of +2.6
# little_miss_sunshine annotation has delay of -8.8
# usar o dataset de palasvras p acertar a sincronizacao do tempo

In [ ]:
movie = VideoFileClip("the_prestige.mkv")
faces_times = pd.read_csv('the_prestige_faces.txt', sep=':', header=0, names=['onset', 'duration'])
words_times = pd.read_csv('the_prestige_words.csv', sep=',', header=0, names=['subtitle', 'start_time_new', 'end_time_new', 'interval_new', 'type'])

faces_times['onset_old'] = faces_times['onset'] 
faces_times['onset'] = faces_times['onset_old'] -8.8
faces_times = faces_times[faces_times['onset']>0].reset_index(drop=True)
faces_times = faces_times[['onset','duration']]
faces_times

In [ ]:
words_times

In [ ]:
movie.duration

### 2. Get clips (images and audio) with faces  

#### Functions

In [ ]:
def extract_audio_and_image(movie, start_time, end_time):
    video = movie.subclip(start_time, end_time)
    images = movie.subclip(start_time, end_time).without_audio()
    audio = movie.audio.subclip(start_time, end_time)
    return video, images, audio
    

def get_clips(movie, times, label):
    clips = {}
    for index, times in tqdm(times.iterrows(), total=times.shape[0], desc="Extracting clips"):
        onset_time = times['onset']
        duration = times['duration']
        end_time = onset_time + duration
        video, images, audio = extract_audio_and_image(movie, float(onset_time), float(end_time))
        clips_data = {
            'onset': onset_time,
            'duration': duration,
            'label': label,
            'video': video,
            'audio': audio,
            'images': images
        }
        clips[index] = clips_data
    return clips


def get_inverted_times(movie, times, epsilon = 0.1, threshold = 1):
    # getting inverted times
    df = faces_times.copy()
    df['onset_end'] = df['onset'] + df['duration'] 
    df['offset_start'] = df['onset'] + df['duration'] + epsilon
    df['offset_end'] = pd.concat([df['onset'].iloc[1:], pd.Series(movie.duration)], ignore_index=True) - epsilon 
    df['offset_duration'] = df['offset_end'] - df['offset_start'] 
    first_row = pd.DataFrame({'onset': [np.nan], 'duration': [np.nan], 'onset_end': [np.nan],
                              'offset_start': [0], 'offset_duration':df.iloc[0,0]- epsilon , 'offset_end':df.iloc[0,0]- epsilon})
    df = pd.concat([first_row, df], ignore_index=True).round(1)
    # new
    inverted_times = df[['offset_start','offset_duration']]
    inverted_times.rename(columns={'offset_start':'onset','offset_duration':'duration'}, inplace = True)
    # drop durations too small
    inverted_times = inverted_times[(inverted_times.duration > threshold)].reset_index(drop=True)
    return inverted_times


def preview_movie(clip):
    temp_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
    clip.write_videofile(temp_file.name, codec="libx264")
    clip_final = Video(temp_file.name, embed=True)
    return clip_final

#### With Faces

In [ ]:
clips_with_faces = get_clips(movie, times=faces_times[:300], label='has_faces')

In [ ]:
clips_with_faces[index]

In [ ]:
index = 0
print(clips_with_faces[index])
preview_movie(clips_with_faces[index]['video'])

In [ ]:
clips = [clip['video'] for clip in clips_with_faces.values()]
concat_clips = concatenate_videoclips(clips)
preview_movie(concat_clips)

#### Without Faces

In [ ]:
times_without_faces = get_inverted_times(movie, faces_times, epsilon = 0.1, threshold = 1)
times_without_faces

In [ ]:
clips_without_faces = get_clips(movie, times=times_without_faces[:-1], label='no_faces')

In [ ]:
index = 1
print(clips_without_faces[index])
preview_movie(clips_without_faces[index]['video'])

In [ ]:
clips = [clip['video'] for clip in clips_without_faces.values()]
concat_clips = concatenate_videoclips(clips)
preview_movie(concat_clips)